In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('train_LZdllcl.csv')

In [3]:
test=pd.read_csv('test_2umaH9m.csv')

In [4]:
test.isnull().sum().sort_values(ascending=False)

previous_year_rating    1812
education               1034
avg_training_score         0
awards_won?                0
KPIs_met >80%              0
length_of_service          0
age                        0
no_of_trainings            0
recruitment_channel        0
gender                     0
region                     0
department                 0
employee_id                0
dtype: int64

In [5]:
train.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [6]:
train.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')

In [7]:
train.is_promoted.value_counts()

0    50140
1     4668
Name: is_promoted, dtype: int64

In [8]:
train.is_promoted.value_counts()[1]/(train.is_promoted.value_counts()[0]+train.is_promoted.value_counts()[1])

0.08517004816815063

####  Unbalanced dataset 8.5% of data is for people getting promoted

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score

In [10]:
train.isnull().sum().sort_values(ascending=False)

previous_year_rating    4124
education               2409
is_promoted                0
avg_training_score         0
awards_won?                0
KPIs_met >80%              0
length_of_service          0
age                        0
no_of_trainings            0
recruitment_channel        0
gender                     0
region                     0
department                 0
employee_id                0
dtype: int64

In [11]:
train['previous_year_rating'].fillna(train['previous_year_rating'].median(),inplace=True)

In [12]:
train['education'].fillna(train['education'].mode()[0],inplace=True)

In [13]:
train.isnull().sum().sort_values(ascending=False)

is_promoted             0
avg_training_score      0
awards_won?             0
KPIs_met >80%           0
length_of_service       0
previous_year_rating    0
age                     0
no_of_trainings         0
recruitment_channel     0
gender                  0
education               0
region                  0
department              0
employee_id             0
dtype: int64

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
employee_id             54808 non-null int64
department              54808 non-null object
region                  54808 non-null object
education               54808 non-null object
gender                  54808 non-null object
recruitment_channel     54808 non-null object
no_of_trainings         54808 non-null int64
age                     54808 non-null int64
previous_year_rating    54808 non-null float64
length_of_service       54808 non-null int64
KPIs_met >80%           54808 non-null int64
awards_won?             54808 non-null int64
avg_training_score      54808 non-null int64
is_promoted             54808 non-null int64
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


In [15]:
train1=train.iloc[:,1:]

In [16]:
chrac=train1.select_dtypes(include=['object']).columns
chrac

Index(['department', 'region', 'education', 'gender', 'recruitment_channel'], dtype='object')

In [17]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X=LabelEncoder()
def label_encode(df,ch):
    for c in ch:
        df.loc[:, c]=labelencoder_X.fit_transform(df.loc[:,c])
    return df

In [18]:
train_p=label_encode(train1,chrac)

In [19]:
train_p.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,7,31,2,0,2,1,35,5.0,8,1,0,49,0
1,4,14,0,1,0,1,30,5.0,4,0,0,60,0
2,7,10,0,1,2,1,34,3.0,7,0,0,50,0
3,7,15,0,1,0,2,39,1.0,10,0,0,50,0
4,8,18,0,1,0,1,45,3.0,2,0,0,73,0


In [20]:
train_p.is_promoted.value_counts()

0    50140
1     4668
Name: is_promoted, dtype: int64

In [21]:
X, y = train_p.loc[:,train_p.columns!='is_promoted'].values, train_p.loc[:,'is_promoted'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=123,stratify=train_p['is_promoted'])

In [22]:
#Oversampling Minority Class
from sklearn.utils import resample


# separate minority and majority classes
not_promoted = train_p[train_p['is_promoted']==0]
promoted = train_p[train_p['is_promoted']==1]

# upsample minority
promoted_upsampled = resample(promoted,
                          replace=True, # sample with replacement
                          n_samples=len(not_promoted), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_promoted, promoted_upsampled])

# check new class counts
upsampled['is_promoted'].value_counts()



1    50140
0    50140
Name: is_promoted, dtype: int64

In [23]:
# trying logistic regression again with the balanced dataset
y_train = upsampled['is_promoted']
X_train = upsampled.drop('is_promoted', axis=1)

# upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

#upsampled_pred = upsampled.predict(X_test)

In [24]:
upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)


In [25]:
upsampled_pred = upsampled.predict(X_test)

In [26]:
pd.DataFrame(confusion_matrix(y_test, upsampled_pred))

,0,1
0,10831,4212
1,406,994


In [27]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [29]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,15040,3
1,0,1400


In [32]:
test['education'].fillna(test['education'].mode()[0],inplace=True)

In [34]:
test['previous_year_rating'].fillna(test['previous_year_rating'].median(),inplace=True)

In [35]:
test.isnull().sum().sort_values(ascending=False)

avg_training_score      0
awards_won?             0
KPIs_met >80%           0
length_of_service       0
previous_year_rating    0
age                     0
no_of_trainings         0
recruitment_channel     0
gender                  0
education               0
region                  0
department              0
employee_id             0
dtype: int64

In [40]:
chw=test.select_dtypes(include=['object']).columns

In [41]:
test_p=label_encode(test,chw)

In [44]:
test.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score'],
      dtype='object')

In [42]:
X_Test=test_p.iloc[:,1:].values

In [49]:
y_pred1 = classifier.predict(X_Test)


In [45]:
sub=pd.DataFrame()

In [46]:
sub['employee_id']=test.loc[:,'employee_id'].values

In [50]:
sub['is_promoted']=y_pred1

In [52]:
sub.to_csv('submission1.csv',index=False)